# Comparison Between TreeValue and Tianshou Batch

In this section, we will take a look at the feature and performance of the [Tianshou Batch](https://github.com/thu-ml/tianshou) library, which is developed by Tsinghua Machine Learning Group.

Before starting the comparison, let us define some thing.

In [1]:
import torch 

_TREE_DATA_1 = {'a': 1, 'b': 2, 'x': {'c': 3, 'd': 4}}
_TREE_DATA_2 = {
    'a': torch.randn(2, 3), 
    'x': {
        'c': torch.randn(3, 4)
    },
}
_TREE_DATA_3 = {
    'obs': torch.randn(4, 84, 84),
    'action': torch.randint(0, 6, size=(1,)),
    'reward': torch.rand(1),
}

/opt/hostedtoolcache/Python/3.8.16/x64/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Read and Write Operation

Reading and writing are the two most common operations in the tree data structure based on the data model (TreeValue and Tianshou Batch both belong to this type), so this section will compare the reading and writing performance of these two libraries.

### TreeValue's Get and Set

In [2]:
from treevalue import FastTreeValue

t = FastTreeValue(_TREE_DATA_2)

In [3]:
t

<FastTreeValue 0x7f1be8059bb0>
├── 'a' --> tensor([[-0.3719, -0.5147,  1.1366],
│                   [ 0.0782,  0.9471,  0.8317]])
└── 'x' --> <FastTreeValue 0x7f1be8059c10>
    └── 'c' --> tensor([[ 0.6118,  1.6741,  0.0199,  0.5970],
                        [-0.9532, -0.5711, -0.0916, -0.0279],
                        [-0.8551,  0.1324,  1.6024,  0.4098]])

In [4]:
t.a

tensor([[-0.3719, -0.5147,  1.1366],
        [ 0.0782,  0.9471,  0.8317]])

In [5]:
%timeit t.a

57.5 ns ± 0.0517 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [6]:
new_value = torch.randn(2, 3)
t.a = new_value

t

<FastTreeValue 0x7f1be8059bb0>
├── 'a' --> tensor([[-1.1428, -0.8127,  0.8222],
│                   [ 0.8049,  0.4354, -1.4085]])
└── 'x' --> <FastTreeValue 0x7f1be8059c10>
    └── 'c' --> tensor([[ 0.6118,  1.6741,  0.0199,  0.5970],
                        [-0.9532, -0.5711, -0.0916, -0.0279],
                        [-0.8551,  0.1324,  1.6024,  0.4098]])

In [7]:
%timeit t.a = new_value

61.5 ns ± 0.0734 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


### Tianshou Batch's Get and Set

In [8]:
from tianshou.data import Batch

b = Batch(**_TREE_DATA_2)

In [9]:
b

Batch(
    a: tensor([[-0.3719, -0.5147,  1.1366],
               [ 0.0782,  0.9471,  0.8317]]),
    x: Batch(
           c: tensor([[ 0.6118,  1.6741,  0.0199,  0.5970],
                      [-0.9532, -0.5711, -0.0916, -0.0279],
                      [-0.8551,  0.1324,  1.6024,  0.4098]]),
       ),
)

In [10]:
b.a

tensor([[-0.3719, -0.5147,  1.1366],
        [ 0.0782,  0.9471,  0.8317]])

In [11]:
%timeit b.a

51.1 ns ± 0.0309 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [12]:
new_value = torch.randn(2, 3)
b.a = new_value

b

Batch(
    a: tensor([[-1.1132,  0.1158, -0.6283],
               [ 1.3235, -0.5010,  0.1394]]),
    x: Batch(
           c: tensor([[ 0.6118,  1.6741,  0.0199,  0.5970],
                      [-0.9532, -0.5711, -0.0916, -0.0279],
                      [-0.8551,  0.1324,  1.6024,  0.4098]]),
       ),
)

In [13]:
%timeit b.a = new_value

496 ns ± 0.0868 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


## Initialization

### TreeValue's Initialization

In [14]:
%timeit FastTreeValue(_TREE_DATA_1)

824 ns ± 0.173 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


### Tianshou Batch's Initialization

In [15]:
%timeit Batch(**_TREE_DATA_1)

11.3 µs ± 35.6 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


## Deep Copy Operation

In [16]:
import copy

### Deep Copy of TreeValue

In [17]:
t3 = FastTreeValue(_TREE_DATA_3)
%timeit copy.deepcopy(t3)

135 µs ± 262 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


### Deep Copy of Tianshou Batch

In [18]:
b3 = Batch(**_TREE_DATA_3)
%timeit copy.deepcopy(b3)

137 µs ± 673 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


## Stack, Concat and Split Operation

### Performance of TreeValue

In [19]:
trees = [FastTreeValue(_TREE_DATA_2) for _ in range(8)]

In [20]:
t_stack = FastTreeValue.func(subside=True)(torch.stack)

t_stack(trees)

<FastTreeValue 0x7f1b3035be20>
├── 'a' --> tensor([[[-0.3719, -0.5147,  1.1366],
│                    [ 0.0782,  0.9471,  0.8317]],
│           
│                   [[-0.3719, -0.5147,  1.1366],
│                    [ 0.0782,  0.9471,  0.8317]],
│           
│                   [[-0.3719, -0.5147,  1.1366],
│                    [ 0.0782,  0.9471,  0.8317]],
│           
│                   [[-0.3719, -0.5147,  1.1366],
│                    [ 0.0782,  0.9471,  0.8317]],
│           
│                   [[-0.3719, -0.5147,  1.1366],
│                    [ 0.0782,  0.9471,  0.8317]],
│           
│                   [[-0.3719, -0.5147,  1.1366],
│                    [ 0.0782,  0.9471,  0.8317]],
│           
│                   [[-0.3719, -0.5147,  1.1366],
│                    [ 0.0782,  0.9471,  0.8317]],
│           
│                   [[-0.3719, -0.5147,  1.1366],
│                    [ 0.0782,  0.9471,  0.8317]]])
└── 'x' --> <FastTreeValue 0x7f1be8026940>
    └── 'c' --> tensor([[[

In [21]:
%timeit t_stack(trees)

32.5 µs ± 124 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [22]:
t_cat = FastTreeValue.func(subside=True)(torch.cat)

t_cat(trees)

<FastTreeValue 0x7f1bbc4a50d0>
├── 'a' --> tensor([[-0.3719, -0.5147,  1.1366],
│                   [ 0.0782,  0.9471,  0.8317],
│                   [-0.3719, -0.5147,  1.1366],
│                   [ 0.0782,  0.9471,  0.8317],
│                   [-0.3719, -0.5147,  1.1366],
│                   [ 0.0782,  0.9471,  0.8317],
│                   [-0.3719, -0.5147,  1.1366],
│                   [ 0.0782,  0.9471,  0.8317],
│                   [-0.3719, -0.5147,  1.1366],
│                   [ 0.0782,  0.9471,  0.8317],
│                   [-0.3719, -0.5147,  1.1366],
│                   [ 0.0782,  0.9471,  0.8317],
│                   [-0.3719, -0.5147,  1.1366],
│                   [ 0.0782,  0.9471,  0.8317],
│                   [-0.3719, -0.5147,  1.1366],
│                   [ 0.0782,  0.9471,  0.8317]])
└── 'x' --> <FastTreeValue 0x7f1be80bad30>
    └── 'c' --> tensor([[ 0.6118,  1.6741,  0.0199,  0.5970],
                        [-0.9532, -0.5711, -0.0916, -0.0279],
                 

In [23]:
%timeit t_cat(trees)

30.8 µs ± 93.7 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [24]:
t_split = FastTreeValue.func(rise=True)(torch.split)
tree = FastTreeValue({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1),
})

%timeit t_split(tree, 1)

58.6 µs ± 475 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


### Performance of Tianshou Batch

In [25]:
batches = [Batch(**_TREE_DATA_2) for _ in range(8)]

Batch.stack(batches)

Batch(
    a: tensor([[[-0.3719, -0.5147,  1.1366],
                [ 0.0782,  0.9471,  0.8317]],
       
               [[-0.3719, -0.5147,  1.1366],
                [ 0.0782,  0.9471,  0.8317]],
       
               [[-0.3719, -0.5147,  1.1366],
                [ 0.0782,  0.9471,  0.8317]],
       
               [[-0.3719, -0.5147,  1.1366],
                [ 0.0782,  0.9471,  0.8317]],
       
               [[-0.3719, -0.5147,  1.1366],
                [ 0.0782,  0.9471,  0.8317]],
       
               [[-0.3719, -0.5147,  1.1366],
                [ 0.0782,  0.9471,  0.8317]],
       
               [[-0.3719, -0.5147,  1.1366],
                [ 0.0782,  0.9471,  0.8317]],
       
               [[-0.3719, -0.5147,  1.1366],
                [ 0.0782,  0.9471,  0.8317]]]),
    x: Batch(
           c: tensor([[[ 0.6118,  1.6741,  0.0199,  0.5970],
                       [-0.9532, -0.5711, -0.0916, -0.0279],
                       [-0.8551,  0.1324,  1.6024,  0.4098]],
         

In [26]:
%timeit Batch.stack(batches)

79.2 µs ± 164 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [27]:
Batch.cat(batches)

Batch(
    a: tensor([[-0.3719, -0.5147,  1.1366],
               [ 0.0782,  0.9471,  0.8317],
               [-0.3719, -0.5147,  1.1366],
               [ 0.0782,  0.9471,  0.8317],
               [-0.3719, -0.5147,  1.1366],
               [ 0.0782,  0.9471,  0.8317],
               [-0.3719, -0.5147,  1.1366],
               [ 0.0782,  0.9471,  0.8317],
               [-0.3719, -0.5147,  1.1366],
               [ 0.0782,  0.9471,  0.8317],
               [-0.3719, -0.5147,  1.1366],
               [ 0.0782,  0.9471,  0.8317],
               [-0.3719, -0.5147,  1.1366],
               [ 0.0782,  0.9471,  0.8317],
               [-0.3719, -0.5147,  1.1366],
               [ 0.0782,  0.9471,  0.8317]]),
    x: Batch(
           c: tensor([[ 0.6118,  1.6741,  0.0199,  0.5970],
                      [-0.9532, -0.5711, -0.0916, -0.0279],
                      [-0.8551,  0.1324,  1.6024,  0.4098],
                      [ 0.6118,  1.6741,  0.0199,  0.5970],
                      [-0.9532, -

In [28]:
%timeit Batch.cat(batches)

143 µs ± 158 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [29]:
batch = Batch({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1)}
)

%timeit list(Batch.split(batch, 1, shuffle=False, merge_last=True))

308 µs ± 4.22 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)
